## Installing and importing necessary libraries

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import pandas as pd

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00


In [ ]:
pip install isic-cli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.1 MB/s eta 0:00:00


### Loading the dataset

Here we are loading a part of the ISIC dataset.

In [ ]:
!isic image download --search 'age_approx:[25 TO 35]' images/

Psst, you're logged out. Logging in with `isic user login` might return more data.


Successfully downloaded 15,136 images to images_50/.
Successfully wrote 15,136 metadata records to images_50/metadata.csv.
Successfully wrote attributions to images_50/attribution.txt.
Successfully wrote 3 license(s) to images_50/licenses.


### Some data exploration

In [ ]:
metadata = pd.read_csv("/content/images_50/metadata.csv")

In [ ]:
metadata.columns

Index(['isic_id', 'attribution', 'copyright_license', 'acquisition_day',
       'age_approx', 'anatom_site_general', 'benign_malignant',
       'clin_size_long_diam_mm', 'dermoscopic_type', 'diagnosis',
       'diagnosis_confirm_type', 'family_hx_mm', 'image_type', 'lesion_id',
       'mel_class', 'mel_mitotic_index', 'mel_thick_mm', 'mel_type',
       'mel_ulcer', 'melanocytic', 'nevus_type', 'patient_id',
       'personal_hx_mm', 'pixels_x', 'pixels_y', 'sex'],
      dtype='object')

In [ ]:
metadata.isna().sum()

isic_id                       0
attribution                   0
copyright_license             0
acquisition_day            7944
age_approx                    0
anatom_site_general        1604
benign_malignant              0
clin_size_long_diam_mm    13127
dermoscopic_type           9029
diagnosis                     8
diagnosis_confirm_type      254
family_hx_mm              10986
image_type                    0
lesion_id                  1427
mel_class                 13716
mel_mitotic_index         13896
mel_thick_mm              13789
mel_type                  13857
mel_ulcer                 13882
melanocytic                6619
nevus_type                13658
patient_id                 4453
personal_hx_mm            10914
pixels_x                      0
pixels_y                      0
sex                           7
dtype: int64

In [ ]:
for col in metadata.columns:
  print(col,"*****************")
  print(metadata[col].unique())
  print()

In [ ]:
cols_to_use = ['benign_malignant',
       'clin_size_long_diam_mm', 'dermoscopic_type', 'diagnosis', 'image_type',
       'melanocytic', 'nevus_type']

In [ ]:
dic = dict(metadata.loc[0][cols_to_use].dropna())

### GPT-3.5-turbo

In [ ]:
import openai
from openai import OpenAI
import pandas as pd
from google.colab import userdata

GPT_API_KEY = userdata.get('GPT')

# Function to translate a single sentence
client = OpenAI(
  api_key=GPT_API_KEY,
)


def generate_gpt(prompt):
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )

    return completion.choices[0].message.content

In [ ]:
a = generate_gpt(get_prompt(dic))

In [ ]:
eval(a)['data']

[{'question': 'What is the diagnosis of the skin condition in the image?',
  'answer': 'The diagnosis of the skin condition in the image is nevus.'},
 {'question': 'Is the skin condition benign or malignant?',
  'answer': 'The skin condition in the image is benign.'},
 {'question': 'What type of dermoscopy was used for imaging the lesion?',
  'answer': 'The type of dermoscopy used was contact polarized dermoscopy.'},
 {'question': 'Can you provide more information about the nevus type seen in the image?',
  'answer': 'Unfortunately, the nevus type is not specified in the provided information.'}]

## VQA dataset

In [ ]:
import pandas as pd
import PIL
import datasets
from datasets import Dataset, load_dataset, Image
from tqdm import tqdm

### Generating the dataset

In [ ]:
def get_prompt(dic):
  Prompt = f"""
  Generate question answer pairs based on the follwing dictionary: {dic}
  here are the meanings of the fields:
  'age_approx': The approximate age of the patient at the time the image was taken.
  'anatom_site_general': Describes the general anatomical site on the body where the lesion or skin condition is located.
  'benign_malignant': Indicates whether the lesion or skin condition is benign (non-cancerous) or malignant (cancerous).
  'clin_size_long_diam_mm': Clinical size of the lesion, measured as the longest diameter in millimeters.
  'dermoscopic_type': Describes the type of dermoscopy used for imaging the lesion or skin condition.
  'diagnosis': The diagnosis or classification of the skin condition.
  'image_type': Describes the type of image.
  'melanocytic': Indicates whether the lesion is melanocytic (related to melanocytes, cells that produce pigment).
  'nevus_type': Describes the type of nevus (mole) if applicable.

  Your job is to simulate a plausible conversation between a doctor and an AI medical assistant, so you can add additional question and answers pairs (for example about the meaning of the diagnosis).
  If a field is empty in the provided dictionary (containing 'nan' for example) don't ask about it.
  The assitant should talk as if it bases its answer on an image provided by the doctor.

  The output should be JSON with one field called 'data' containing a list of question answer pairs in JSON format.
  """
  return Prompt

In [ ]:
feature = datasets.Image()

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['id', 'image', 'conversation'])

for i in tqdm(range(1, 5)):
  image_id = metadata["isic_id"].iloc[i]
  image_path = f"images/{image_id}.JPG"

  # Open image with Pillow
  with PIL.Image.open(image_path) as img:
      # Convert image to base64
      img_data = img.copy()

  dic = dict(metadata.loc[0][cols_to_use].dropna())
  response = generate_gpt(get_prompt(dic))
  conversation = eval(response)
  # Append data to the DataFrame
  df = df.append({
      "id": i+1,
      "image": feature.encode_example(img_data),
      "conversation": conversation
  }, ignore_index=True)

  if i%100 == 0:
    df.to_csv(f"/content/drive/MyDrive/Winter school/output_{i//100}.csv", index=False)
    df = pd.DataFrame(columns=['id', 'image', 'conversation'])

100%|██████████| 4/4 [00:13<00:00,  3.27s/it]


### Uploading the dataset

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
dss = []
json_columns = ['image', 'conversation']
for i in tqdm(range(1,8)):
  df = pd.read_csv(f"/content/drive/MyDrive/Winter school/output_{i}.csv")
  for col in json_columns:
      df[col] = df[col].apply(eval)
  ds = Dataset.from_pandas(df).cast_column('image', Image(decode=True))
  dss.append(ds)

100%|██████████| 7/7 [02:48<00:00, 24.00s/it]


In [ ]:
dataset = datasets.concatenate_datasets(dss)

In [ ]:
dataset = Dataset.from_pandas(df).cast_column('image', Image(decode=True))

print(dataset)

Dataset({
    features: ['id', 'image', 'conversation'],
    num_rows: 700
})


In [ ]:
from huggingface_hub import login
login()

In [ ]:
dataset.push_to_hub("WinterSchool/ISIC25TO35")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ?it/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/WinterSchool/ISIC25TO35/commit/dc4afcb29260b151cc8f5163236c0df156b9875c', commit_message='Upload dataset', commit_description='', oid='dc4afcb29260b151cc8f5163236c0df156b9875c', pr_url=None, pr_revision=None, pr_num=None)

## Model calling dataset

In [ ]:
import pandas as pd
import PIL
import datasets
from datasets import Dataset, load_dataset, Image
from tqdm import tqdm

In [ ]:
base_prompt=f"""
### Instructions
Your job is to simulate a conversation between a doctor and an AI medical assistant. The doctor provided the AI
assistant with an image and started asking questions about it.
You will be given 'input' giving you information about the image.

The medical assistant has access to the following models that it can call to perform some computer vision tasks
such as classification and object detection.
Here are the model that the AI assistant can use:
{{
 'name': 'skin_cond_classifier',
 'description': 'Provide skin condition diagnosis',
 'type':'classifier'
 'classes':{{'malignant', 'benign', 'indeterminate/malignant','indeterminate/benign', 'indeterminate'}}
}}

To use these functions The assistant respond like so:
<modelcall> {{'name': 'model_name'}} </modelcall>

The assitant should talk as if it bases its answer on an image provided by the doctor.
Try to make the conversation convincing, you can add additional question and answers pairs (for example about the meaning of something, or the next course of action).
In the simulated conversation, you should also include question/answer pairs where the doctor ask about something
that the AI assistant can't call a model for and others that can be answered from the input.



Your output should be a JSON object with one field called 'data' containing a list of JSON objects in the form of {{ 'from': , 'value': }}.

Here is an example of a plausible conversation:
{{ 'from': 'doctor', 'value': 'Is diabetic retinopathy present in this image?' }}
{{ 'from': 'AI', 'value': '<modelcall> {{'name': 'Diab_Retinopathy_Detector'</modelcall>' }}
{{ 'from': 'function_response', 'value': '{{'malignant': 0.8,'benign': 0.2,'indeterminate/malignant': 0.3,'indeterminate/benign': 0.1,'indeterminate': 0}}' }}
{{ 'from': 'AI', 'value': 'No the eye show little sign of diabetic retinopathy' }}

Remember both the doctor and AI assistant don't have access to this input. They only have access to the image described by this input.

### Input

"""


In [ ]:
def get_prompt1(dic):
  var_prompt=f"""
{dic}
here are the meanings of the fields:
'benign_malignant': Indicates whether the lesion or skin condition is benign (non-cancerous) or malignant (cancerous).
'clin_size_long_diam_mm': Clinical size of the lesion, measured as the longest diameter in millimeters.
'dermoscopic_type': Describes the type of dermoscopy used for imaging the lesion or skin condition.
'diagnosis': The diagnosis or classification of the skin condition.
'image_type': Describes the type of image.
'melanocytic': Indicates whether the lesion is melanocytic (related to melanocytes, cells that produce pigment).
'nevus_type': Describes the type of nevus (mole) if applicable.
"""
  return base_prompt + var_prompt

In [1]:
feature = datasets.Image()

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['id', 'image', 'conversation'])

for i in tqdm(range(1, 501)):
  image_id = metadata["isic_id"].iloc[i]
  image_path = f"images/{image_id}.JPG"

  # Open image with Pillow
  with PIL.Image.open(image_path) as img:
      # Convert image to base64
      img_data = img.copy()

  dic = dict(metadata.loc[0][cols_to_use].dropna())
  response = generate_gpt(get_prompt1(dic))
  conversation = eval(response)
  # Append data to the DataFrame
  df = df.append({
      "id": i+1,
      "image": feature.encode_example(img_data),
      "conversation": conversation
  }, ignore_index=True)

  if i%100 == 0:
    df.to_csv(f"/content/drive/MyDrive/Winter school/output_dc_{i//100}.csv", index=False)
    df = pd.DataFrame(columns=['id', 'image', 'conversation'])